In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.model_selection import train_test_split

from utils.data import get_dataset_paths
from utils.data import Marconi100Dataset
from utils.data import UnfoldedDataset
from utils.training import training_loop

from algos.deep_fib.data import get_masks
from algos.deep_fib.core import DeepFIBEngine
from algos.deep_fib.sci_net import SCINet

In [3]:
paths = get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.1, random_state=42)


In [4]:
m_data_train = Marconi100Dataset(train[:1], scaling="minmax")
m_data_test = Marconi100Dataset(test[:5], scaling="minmax")


Loading: 100%|██████████| 5/5 [00:08<00:00,  1.62s/it]


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

horizon = 1024
stride = 1000
n_masks = 100

batch_size = 32

num_encoder_levels = 2

lr = 1e-3
num_epochs = 5

hidden = None
input_dim = 460
hidden_size = 4
kernel_size = 3
dropout = 0.5

anomaly_threshold = 0.01


cpu


In [8]:
dataset_train = UnfoldedDataset(
    m_data_train, horizon=horizon, stride=stride
)
dataset_test = UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)

masks = get_masks(horizon, n_masks).float()

len(dataset_train), len(dataset_test), masks.size()


(13, 72, torch.Size([100, 1024, 460]))

In [7]:
train_loader = DataLoader(
    dataset_train,
    batch_size,
    shuffle=True,
)
test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
)
len(train_loader), len(test_loader)


(1, 3)

In [8]:
model = SCINet(
    output_len=horizon,
    input_len=horizon,
    num_encoder_levels=num_encoder_levels,
    hidden_decoder_sizes=hidden,
    input_dim=input_dim,
    hidden_size=hidden_size,
    kernel_size=kernel_size,
    dropout=dropout,
).float()

engine = DeepFIBEngine(anomaly_threshold, masks)

optim = Adam(model.parameters(), lr=lr)
lr_sched = CosineAnnealingLR(optim, num_epochs)

In [10]:
training_loop(
    model=model,
    engine=engine,
    num_epochs=num_epochs,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    device=device,
    optimizer=optim,
    lr_scheduler=lr_sched,
)


Test 0:  33%|███▎      | 1/3 [00:01<00:02,  1.05s/it] 

torch.Size([32, 1024]) torch.int32 torch.Size([32, 1024]) torch.int32
tensor(0.9789, device='cuda:0')


Test 0:  67%|██████▋   | 2/3 [00:02<00:01,  1.03s/it]

torch.Size([32, 1024]) torch.int32 torch.Size([32, 1024]) torch.int32
tensor(0.9864, device='cuda:0')


torch.Size([8, 1024]) torch.int32 torch.Size([8, 1024]) torch.int32
tensor(0.9775, device='cuda:0')
Epoch 0 - train_loss = 0.137 - test_loss = 0.143 - lr = 0.00e+00
loss {'test_mean': 0.14330251018206278} 1
f1 {'test_mean': 0.9809468587239584} 1
mre {'test_mean': 0.14330251018206278} 1


...

Test 4:  33%|███▎      | 1/3 [00:01<00:02,  1.04s/it] 

torch.Size([32, 1024]) torch.int32 torch.Size([32, 1024]) torch.int32
tensor(0.9789, device='cuda:0')


Test 4:  67%|██████▋   | 2/3 [00:02<00:01,  1.07s/it]

torch.Size([32, 1024]) torch.int32 torch.Size([32, 1024]) torch.int32
tensor(0.9864, device='cuda:0')


torch.Size([8, 1024]) torch.int32 torch.Size([8, 1024]) torch.int32
tensor(0.9775, device='cuda:0')
Epoch 4 - train_loss = 0.107 - test_loss = 0.111 - lr = 9.05e-04
loss {'test_mean': 0.1114923506975174} 5
f1 {'test_mean': 0.9809468587239584} 5
mre {'test_mean': 0.1114923506975174} 5
